# 深層学習ノートブック-3 pytorchによる多項ロジスティック回帰
参考：  
* [行列積](https://w3e.kanazawa-it.ac.jp/math/category/gyouretu/senkeidaisu/henkan-tex.cgi?target=/math/category/gyouretu/senkeidaisu/gyouretu-no-seki.html)

pytorchのAutograd, Tensorを用いて多項ロジスティック回帰による分類をスクラッチで実装する。  
ここで実装する一連の流れは深層学習におけるモデル学習プロセスの基礎であり、非常に重要。  

ここではMNIST（手書き数字(0~9)）データセットを用いた10クラス分類のタスクを多項ロジスティック回帰で解くことを考える。  
MNISTデータセットにある各pixel値(8x8=64)を特徴量として扱う。  

データ数をm、特徴量の行列を$\bm{X}$、$\bm{X}$に対応する重み行列を$W^T$※、バイアス項の行列を$\bm{b}$とすると、  
softmax関数への入力$\bm{z}$は下記のように表せる。（pytorchではこの形を採用している。）  
※行列積の形で表わすために転置をとっている。


$\bm{z} = \bm{XW}^T+\bm{b} $  
$=\left(
\begin{matrix} 
x_{1,1} & x_{1,2} & ... & x_{1,64}\\ 
x_{2,1} & x_{2,2} & ... & x_{2, 64}\\
. & . & ... & .\\
x_{m,1} & x_{m,2} & ... & x_{m, 64}
\end{matrix} 
\right)$
$\left(
\begin{matrix} 
w_{1,1} & w_{1,2} & ... & w_{1,10}\\ 
w_{2,1} & w_{2,2} & ... & x_{2,10}\\
. & . & ... & .\\
w_{64,1} & x_{64,2} & ... & x_{64, 10}
\end{matrix} 
\right)$
$+\left(
\begin{matrix} 
b_{1} & b_{2} & ... & b_{10}\\ 
b_{1} & b_{2} & ... & b_{10}\\
. & . & ... & .\\
b_{1} & b_{2} & ... & b_{10}
\end{matrix}
\right)$  

$=\left(
\begin{matrix} 
\sum_{k=1}^{64} x_{1, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{1, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{1, k}w_{k, 10}+b_{10}\\ 
\sum_{k=1}^{64} x_{2, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{2, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{2, k}w_{k, 10}+b_{10}\\ 
. & . & ... & .\\
\sum_{k=1}^{64} x_{m, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{m, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{m, k}w_{k, 10}+b_{10}\\ 
\end{matrix} 
\right)$

$=\left(
\begin{matrix} 
z_{1,1} & z_{1,2} & ... & z_{1,10}\\ 
z_{2,1} & z_{2,2} & ... & z_{2,10}\\
. & . & ... & .\\
z_{m,1} & z_{m,2} & ... & z_{m, 10}
\end{matrix} 
\right)$

$\bm{X}$のshapeはデータ数m × 特徴量数64  
$\bm{W}^T$は重みづけの対象パラメータ数（すなわち特徴量数64）× 最終的な出力列数（10クラス分類なので10）※、  
$\bm{b}$は1 × 最終的な出力列数となるが、上記ではbroadcastされた状態で書いている。  
最終的に$\bm{z}$は各データ数 × クラス数というshapeで表わされ、クラスごとの線形回帰の結果が各要素に対応する。  

※転置前の$\bm{W}$は10*64

# 前処理・初期化

In [64]:
import sklearn
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。

In [88]:
# 変数定義
learning_rate = 0.03
loss_log = []  #損失記録用のリスト

In [86]:
# データロード
dataset = sklearn.datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'])
y_true = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of Y: {Y.shape}')
print(Y[1])

shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.], dtype=torch.float64)
shape of Y: torch.Size([1797, 10])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])


datasetのimagesは1797 x 8 x 8の形式になっており、reshapeする必要があるため、'data'から読み込んだ方が楽。

In [87]:
# 目的変数のエンコーディング
y_true = F.one_hot(y_true, num_classes=10)
print(f'shape of Y: {y_true.shape}')
print(y_true)

shape of Y: torch.Size([1797, 10])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0]])


In [82]:
# 学習データの標準化（ピクセル値を標準化）
for i in range(X.shape[1]):
    mean = X[:, i].mean()
    std = X[:, i].std()
    if(std == 0.):
        X[:, i] = 0.
    else:
        X[:, i] = (X[:, i] - mean) / std


pytorchには標準で標準化のためのクラスがないので自前で実装する必要あり。  
※同じようなスケールのピクセル値の列なので、全列まとめて平均とって標準偏差で割るでも大差ないかも。

In [90]:
# 重みW^T、バイアス項bの初期化
W = torch.rand(size=(10, 64) ,requires_grad=True) #出力×入力
b = torch.rand(size=(1, 10), requires_grad=True) # 1 x 出力

print(W.shape)
print(W[1])
print()
print(b)

torch.Size([10, 64])
tensor([0.8464, 0.9092, 0.3408, 0.4982, 0.5163, 0.9932, 0.5947, 0.5541, 0.0591,
        0.3156, 0.8722, 0.9538, 0.3694, 0.3806, 0.9416, 0.4064, 0.9667, 0.9310,
        0.9185, 0.4682, 0.8012, 0.6893, 0.2805, 0.0374, 0.9744, 0.3576, 0.9610,
        0.6810, 0.3514, 0.9300, 0.6123, 0.3133, 0.1097, 0.3547, 0.0034, 0.9448,
        0.9941, 0.0708, 0.4644, 0.0984, 0.3339, 0.3055, 0.3337, 0.6091, 0.4175,
        0.5785, 0.8451, 0.1898, 0.0282, 0.9654, 0.8262, 0.2924, 0.2932, 0.8604,
        0.4287, 0.3145, 0.0717, 0.2354, 0.7837, 0.2466, 0.5344, 0.4616, 0.7356,
        0.2248], grad_fn=<SelectBackward0>)

tensor([[0.5165, 0.1129, 0.7141, 0.1706, 0.0847, 0.3603, 0.1588, 0.1910, 0.8762,
         0.1424]], requires_grad=True)


# softmax関数・損失関数実装